# LIBRERIES

In [1]:
#pip install pyEDFlib

In [1]:
import numpy as np
import pyedflib
import pandas as pd 
import scipy.signal
from sklearn.model_selection import train_test_split
import os.path
import pickle


# Functions

* Get the EDF files from a folder 

In [2]:
import os
import re
def filenames(path):# createing list of all file paths
    #path='C:/Users/inesb/OneDrive/Desktop/dt'
    filenames_list = os.listdir(path=path) ## list of file names in the directory
    txt = ".txt"
    edf=".edf"
    s=0
    while(s<len(filenames_list)):
        s=0
        for x in filenames_list:
            if(not re.search(".*txt$",x))and(not re.search(".*edf$",x)):
                filenames_list=filenames_list+[x+'/'+i for i in os.listdir(path+'/'+x)]
                filenames_list.remove(x)
            else:
                s=s+1
    l=[path+'/'+i for i in filenames_list if re.search(".*edf$",i)]
    return(l)
    

* Get the CSV files from a folder 

In [3]:
import os
import re
def filenames_csv(path):# createing list of all file paths
    #path='C:/Users/inesb/OneDrive/Desktop/dt'
    filenames_list = os.listdir(path=path) ## list of file names in the directory
    txt = ".csv"
    s=0
    while(s<len(filenames_list)):
        s=0
        for x in filenames_list:
            if(not re.search(".*csv$",x)):
                filenames_list=filenames_list+[x+'/'+i for i in os.listdir(path+'/'+x)]
                filenames_list.remove(x)
            else:
                s=s+1
    l=[path+'/'+i for i in filenames_list if re.search(".*csv$",i)]
    return(l)
    

In [4]:
def tuh_data_load(data_file, selected_channels=[]):

    try:
        f = pyedflib.EdfReader(data_file)
        channel_names = f.getSignalLabels()
        channel_freq = f.getSampleFrequencies()
        
        if len(selected_channels) == 0:
            selected_channels = channel_names

        sigbufs = np.zeros((f.getNSamples()[0],len(selected_channels)))
        for i, channel in enumerate(selected_channels):
            try:
              sigbufs[:, i] = f.readSignal(channel_names.index(channel))
            
            except:
              ValueError
             
              sigbufs[:, i] = np.nan


        df = pd.DataFrame(sigbufs, columns = selected_channels)#.astype('float32')
        '''
        index_increase = np.linspace(0,
                                      len(df)/channel_freq[0],
                                      len(df), endpoint=False)

       

        seconds = index_increase
        
        df['Time'] = seconds
        df = df.set_index('Time')
        df.columns.name = 'Channel'
        '''
       
        return df, channel_freq[0]

    except:
        OSError
        return pd.DataFrame(), None


In [33]:
def final_df(data):
    # renommer les colonnes pour enlever 'eeg' ,'ref' et 'le'
    new_column_list = [x.replace('-REF', '').replace('-LE', '').replace('EEG ', '') for x in data]
    d={}
    for i in range(len(new_column_list)):
        d[data.columns[i]]=new_column_list[i]
    data.rename(columns = d, inplace = True)
    #list des electrodes qu'on va garder
    electrode_list=['FP1','FP2','F3','F4','C3','C4','P3','P4','O1','O2','F7','F8','T3','T4','T5','T6','A1','A2','FZ','CZ','PZ','T1','T2']
    i=0
    dt=pd.DataFrame()
    for x in electrode_list:
        if(x in data.columns):
            dt[x]=data[x]
        else:
            dt[x]=pd.Series(data.mean(axis=1), data.index)
    return dt

In [43]:
def id_session(x): #prend un filename
    return x.split('/')[-1].split('_')[0], x.split('/')[-1].split('_')[1] #retourne un tuple de l'id du patient et la session
def id_file(x):
    return x.split('/')[-1].split('.')[0] #retourne un tuple de l'id du patient et la session
    
def session(a):  #prend un filename
    k=[]
    for x in l:
        id_x=id_session(x)[0]
        session_x=id_session(x)[1]
        id_a=id_session(a)[0]
        session_a=id_session(a)[1]
        if(id_x==id_a)and(session_x==session_a):
            k.append(x)
    return(k) #retourne la list des filenames pour le mm patient dans la mm date
def drop_list(list,l): 
    for x in list:
        if(x in l):
            l.remove(x) 
def list_of_sessions(l): #prend la liste des filenames
    s=[]
    while(len(l)>0):
        k=session(l[0])
        drop_list(k,l)
        s.append(k)
    return(s) #retourne list de list grouped by session 
def list_size_of_sessions(list): #prend unelist des list LL
    t=[]
    for x in list:
        t.append(size_of_session(x))
    return(t) #retourne la list des taille des sessions
def size_of_session(list): #prend une list des fichier pour une seul session
    s=0
    for x in list:
        f=pyedflib.EdfReader(x)   
        s=s+f.getFileDuration()
    return(s) #retourne la taille de la session

In [7]:
def to_csv_session(data,file,destination_diractory):#converting all edf files in filenames to csv files
    data.to_csv('C:/Users/Bolbol/Desktop/csv/csv1/'+destination_diractory+'/'+id_session(file)[0]+'_'+id_session(file)[1]+'.csv',index=False)

In [45]:
def to_csv_session(data,file,destination_diractory):#converting all edf files in filenames to csv files
    data.to_csv('C:/Users/Bolbol/Desktop/csv/csv1/'+destination_diractory+'/'+id_file(file)+'.csv',index=False)

* apply the previous methods and make the csv files in the same size by the sampling method

In [8]:
def preprocessing_deep_learning(ll,destination_diractory):
  for j in ll:
    data,freq=tuh_data_load(j[0])
    if len(j)>0:
      for x in j[1:]:
          dt,fq=tuh_data_load(x)
          data=data.append(dt,ignore_index=True)
    data=final_df(data)
    for i in range(data.shape[0]-1,0,-1):
      if (len(data.iloc[i,:].value_counts())==1):
          data.drop([i],axis=0,inplace=True)
      else:
        break
    d={}
    for i in range(len(data.columns)):
        d[i]=data.columns[i]
    data=pd.DataFrame(scipy.signal.resample(data,10000))
    data.rename(columns = d, inplace = True)
    to_csv_session(data,j[0],destination_diractory)
  return(data)

# Upload Data

In [9]:
epil_path='C:/Users/Bolbol/Desktop/edf/epilepsy'
no_epil_path='C:/Users/Bolbol/Desktop/edf/no_epilepsy'
normal_path='C:/Users/Bolbol/Desktop/normal'

In [10]:
l=filenames(no_epil_path) 

In [11]:
ll=[[x] for x in l]

* apply this function on the different folder to generate the CSV files

In [49]:
%%time
preprocessing_deep_learning(ll,'no_epil_csv')

Wall time: 33min 14s


,FP1,FP2,F3,F4,C3,C4,P3,P4,O1,O2,...,T4,T5,T6,A1,A2,FZ,CZ,PZ,T1,T2
0,-6.141774,-8.005050,-0.152656,-5.822752,1.031984,-3.521229,4.754175,-3.187697,7.812343,0.308685,...,-9.987837,3.475613,-6.653595,-8.273686,-8.273686,-3.503099,-1.713277,0.782613,-8.273686,-8.273686
1,4.401485,2.305695,-1.778911,-0.392569,-7.679845,1.679231,-7.454662,0.021965,-11.252880,-3.061188,...,-1.379767,-18.573719,-1.227512,-1.960426,-1.960426,-0.138360,-0.469057,-2.834978,-1.960426,-1.960426
2,0.158966,1.633475,-1.130672,1.844515,-3.591373,1.192188,-4.238808,-1.715628,-8.055500,-3.772870,...,3.010396,-11.751660,0.743218,-2.897226,-2.897226,-0.450880,-1.548168,-2.705763,-2.897226,-2.897226
3,0.712912,4.802733,-4.411403,5.600605,-2.095665,4.220297,0.665957,-0.450760,-0.930923,-2.609490,...,9.071470,2.937002,4.168023,0.172859,0.172859,-1.103456,-1.690891,-2.313637,0.172859,0.172859
4,3.590340,7.980073,-3.488876,9.346442,-4.127365,8.150362,-2.440509,3.472433,2.771167,2.802155,...,8.431889,2.886276,9.050070,0.332116,0.332116,1.089336,-0.301240,-1.630487,0.332116,0.332116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-6.182387,-8.341379,2.112383,-0.667744,1.750816,1.945701,0.845803,3.174264,2.979063,10.912251,...,4.674353,3.592129,10.902596,-8.053491,-8.053491,-1.894828,-2.365545,1.226179,-8.053491,-8.053491
9996,-7.655952,-9.446406,1.155977,-3.524284,3.077201,0.977843,-0.317678,-3.463768,-9.244469,-4.641768,...,0.123567,0.126575,1.520148,-13.869628,-13.869628,-3.173736,-2.223620,-1.863926,-13.869628,-13.869628
9997,-9.223568,-11.908804,0.024577,-3.076424,-0.850377,-2.054672,-8.677702,-16.310852,-25.445979,-25.374826,...,-12.217477,-11.139822,-15.601005,-22.530938,-22.530938,-1.323135,-1.841170,-7.697954,-22.530938,-22.530938
9998,-14.018738,-15.596463,-2.944443,-5.202387,0.243380,-1.349087,3.646417,1.541602,12.419638,13.956707,...,1.221078,10.055500,9.429634,-15.894972,-15.894972,-6.180766,-1.884439,4.639460,-15.894972,-15.894972


# GENERATING DATA

In [53]:
epil_files=filenames_csv('C:/Users/Bolbol/Desktop/csv/csv1/epil_csv')
no_epil_files=filenames_csv('C:/Users/Bolbol/Desktop/csv/csv1/no_epil_csv')
normal_files=filenames_csv('C:/Users/Bolbol/Desktop/csv/csv1/normal_csv')

* add the values of each csv file in a list and associate a target to it

In [55]:
%%time
data=[]
for i in range(len(epil_files)):
  DATA=pd.read_csv(epil_files[i])
  a=DATA.values
  data.append((a,1))
for i in range(len(no_epil_files)):
  DATA=pd.read_csv(no_epil_files[i])
  a=DATA.values
  data.append((a,0))
for i in range(len(normal_files)):
  DATA=pd.read_csv(normal_files[i])
  a=DATA.values
  data.append((a,0))
X=[]
Y=[]
for x in data:
    X.append(x[0])
    Y.append(x[1])
X=np.array(X)
Y=np.array(Y)
print(X.shape)
print(Y.shape)

(2648, 10000, 23)
(2648,)
Wall time: 2min 19s


* save the variables X et Y in a file using the pickle library

In [56]:
variables = [X,Y]
fichierSauvegarde = open('C:/Users/Bolbol/Desktop/csv/FINAL',"wb")   # path of the destinatuon file
pickle.dump(variables, fichierSauvegarde)
fichierSauvegarde.close()